# Extraction de mots-clés / tags

Un peu de configuration :

In [1]:
LOWERCASE = False

NORMALIZE = None

EXCLUDE_POS = {"ADJ", "ADP", "ADV", "AUX", "DET", "NUM", "VERB", "PUNCT", "SPACE"}

CUSTOM_STOP_WORDS = [
    "auprès",
    "Il",
    "n°",
    "qu'",
    "s'",
    "S'",
    "y",
    "«",
    "»",
    "°",
]

## Extraction des amendements

On commence par configurer Pyramid :

In [2]:
from pyramid.paster import bootstrap

In [3]:
env = bootstrap("development-docker.ini")

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


On charge tous les amendements depuis la base :

In [4]:
from zam_repondeur.models import DBSession, Amendement

In [5]:
amendements = DBSession.query(Amendement).all()

## Nettoyage

On fait une fonction pour nettoyer le HTML :

In [6]:
from bs4 import BeautifulSoup
from textacy import preprocess_text

from zam_repondeur.clean import clean_html

def clean_text(html):
    text = BeautifulSoup(clean_html(html), "lxml").text
    return preprocess_text(text, lowercase=LOWERCASE).replace("’", "'").replace("\n", " ")

In [7]:
amendement = amendements[1000]

In [8]:
amendement.corps

'<p>Avant le 1<sup>er</sup> septembre 2019, le Gouvernement remet au Parlement un rapport portant sur la cr&#233;ation d&#8217;un fonds de modernisation des &#233;tablissements et services priv&#233;s intervenant dans le secteur de la sant&#233; et dans le secteur m&#233;dico-social.</p>'

In [9]:
print(clean_text(amendement.corps))

Avant le 1er septembre 2019, le Gouvernement remet au Parlement un rapport portant sur la création d'un fonds de modernisation des établissements et services privés intervenant dans le secteur de la santé et dans le secteur médico-social.


In [10]:
print(clean_text(amendement.expose))

L'article 40 de la loi n° 2000‑1257 du 23 décembre 2000 de financement de la sécurité sociale a créé le fonds pour la modernisation des établissements de santé publics et privés (FMESPP), ce qui a entraîné la disparition du fonds pour la modernisation des cliniques privées peu après et le transfert de ses obligations et créances vers le FMESPP (cf. art. 26 de la loi n° 2002‑1487 du 20 décembre 2002 de financement de la sécurité sociale pour 2003). Or, le bilan de cette opération de restructuration des fonds de modernisation par le législateur s'avère largement négatif pour les établissements et services privés. En effet, ainsi que le révèle le dernier rapport annuel de gestion du FMESPP (2016) publié par la Caisse des dépôts et des consignations : « depuis 2012, l'action du FMESPP est concentrée sur le financement de mesures nationales, en particulier les investissements et des missions d'expertise au bénéfice des établissements de santé, confiés à l'ATIH et depuis 2013 à l'ASIP ». Cet

In [11]:
print(clean_text(amendement.user_content.comments or ""))

DGOS/1A idem 469


## Spacy

On customise un pipeline français :

In [12]:
import spacy

custom_fr = spacy.load("fr")

for word in CUSTOM_STOP_WORDS:
    lexeme = custom_fr.vocab[word]
    lexeme.is_stop = True

## Textacy

On charge l'exposé des motifs d'un amendement dans un document textacy :

In [13]:
from textacy import Doc

def make_doc(amendement, lang=custom_fr):
    return Doc(clean_text(amendement.expose + "\n\n" + amendement.corps), lang=lang)

In [14]:
clean_text(amendement.expose)

"L'article 40 de la loi n° 2000‑1257 du 23 décembre 2000 de financement de la sécurité sociale a créé le fonds pour la modernisation des établissements de santé publics et privés (FMESPP), ce qui a entraîné la disparition du fonds pour la modernisation des cliniques privées peu après et le transfert de ses obligations et créances vers le FMESPP (cf. art. 26 de la loi n° 2002‑1487 du 20 décembre 2002 de financement de la sécurité sociale pour 2003). Or, le bilan de cette opération de restructuration des fonds de modernisation par le législateur s'avère largement négatif pour les établissements et services privés. En effet, ainsi que le révèle le dernier rapport annuel de gestion du FMESPP (2016) publié par la Caisse des dépôts et des consignations : « depuis 2012, l'action du FMESPP est concentrée sur le financement de mesures nationales, en particulier les investissements et des missions d'expertise au bénéfice des établissements de santé, confiés à l'ATIH et depuis 2013 à l'ASIP ». Ce

In [15]:
doc = make_doc(amendement)
doc

Doc(484 tokens; "L'article 40 de la loi n° 2000‑1257 du 23 décem...")

### Part-of-speech tagging

In [16]:
doc.pos_tagged_text

[[("L'", 'DET'),
  ('article', 'NOUN'),
  ('40', 'NUM'),
  ('de', 'ADP'),
  ('la', 'DET'),
  ('loi', 'NOUN'),
  ('n°', 'NOUN'),
  ('2000‑1257', 'NUM'),
  ('du', 'NOUN'),
  ('23', 'NUM'),
  ('décembre', 'NOUN'),
  ('2000', 'NUM'),
  ('de', 'ADP'),
  ('financement', 'NOUN'),
  ('de', 'ADP'),
  ('la', 'DET'),
  ('sécurité', 'NOUN'),
  ('sociale', 'ADJ'),
  ('a', 'AUX'),
  ('créé', 'VERB'),
  ('le', 'DET'),
  ('fonds', 'NOUN'),
  ('pour', 'ADP'),
  ('la', 'DET'),
  ('modernisation', 'NOUN'),
  ('des', 'DET'),
  ('établissements', 'NOUN'),
  ('de', 'ADP'),
  ('santé', 'NOUN'),
  ('publics', 'ADJ'),
  ('et', 'CCONJ'),
  ('privés', 'ADJ'),
  ('(', 'PUNCT'),
  ('FMESPP', 'PROPN'),
  (')', 'PUNCT'),
  (',', 'PUNCT'),
  ('ce', 'PRON'),
  ('qui', 'PRON'),
  ('a', 'AUX'),
  ('entraîné', 'VERB'),
  ('la', 'DET'),
  ('disparition', 'NOUN'),
  ('du', 'DET'),
  ('fonds', 'NOUN'),
  ('pour', 'ADP'),
  ('la', 'DET'),
  ('modernisation', 'NOUN'),
  ('des', 'DET'),
  ('cliniques', 'ADJ'),
  ('privées', 'A

### Document bag of terms

https://chartbeat-labs.github.io/textacy/api_reference.html#textacy.doc.Doc.to_bag_of_terms

In [17]:
doc.to_bag_of_terms(as_strings=True, filter_nums=True, normalize=NORMALIZE)

{'Caisse des dépôts et des consignations': 1,
 'ATIH': 1,
 'ASIP': 1,
 '–': 1,
 'Gouvernement': 1,
 'Parlement': 1,
 "L'": 1,
 'article': 1,
 'loi': 2,
 '2000‑1257': 1,
 'décembre': 2,
 'financement': 3,
 'sécurité': 2,
 'sociale': 2,
 'créé': 1,
 'fonds': 6,
 'modernisation': 4,
 'établissements': 9,
 'santé': 3,
 'publics': 2,
 'privés': 7,
 'FMESPP': 5,
 'entraîné': 1,
 'disparition': 1,
 'cliniques': 1,
 'privées': 1,
 'transfert': 1,
 'obligations': 1,
 'créances': 1,
 'cf': 1,
 'art': 1,
 '2002‑1487': 1,
 'Or': 1,
 'bilan': 1,
 'opération': 1,
 'restructuration': 2,
 'législateur': 1,
 'avère': 1,
 'largement': 1,
 'négatif': 1,
 'services': 5,
 'En': 1,
 'révèle': 1,
 'rapport': 3,
 'annuel': 2,
 'gestion': 1,
 'publié': 1,
 'Caisse': 1,
 'dépôts': 1,
 'consignations': 1,
 "l'": 5,
 'action': 1,
 'concentrée': 1,
 'mesures': 1,
 'nationales': 1,
 'investissements': 1,
 'missions': 1,
 "d'": 5,
 'expertise': 1,
 'bénéfice': 1,
 'confiés': 1,
 'Cette': 1,
 'orientation': 1,
 'favo

https://chartbeat-labs.github.io/textacy/api_reference.html#textacy.doc.Doc.to_terms_list

In [18]:
list(doc.to_terms_list(as_strings=True, filter_nums=True, normalize=NORMALIZE))

['Caisse des dépôts et des consignations',
 'ATIH',
 'ASIP',
 '–',
 'Gouvernement',
 'Parlement',
 "L'",
 'article',
 'loi',
 '2000‑1257',
 'décembre',
 'financement',
 'sécurité',
 'sociale',
 'créé',
 'fonds',
 'modernisation',
 'établissements',
 'santé',
 'publics',
 'privés',
 'FMESPP',
 'entraîné',
 'disparition',
 'fonds',
 'modernisation',
 'cliniques',
 'privées',
 'transfert',
 'obligations',
 'créances',
 'FMESPP',
 'cf',
 'art',
 'loi',
 '2002‑1487',
 'décembre',
 'financement',
 'sécurité',
 'sociale',
 'Or',
 'bilan',
 'opération',
 'restructuration',
 'fonds',
 'modernisation',
 'législateur',
 'avère',
 'largement',
 'négatif',
 'établissements',
 'services',
 'privés',
 'En',
 'révèle',
 'rapport',
 'annuel',
 'gestion',
 'FMESPP',
 'publié',
 'Caisse',
 'dépôts',
 'consignations',
 "l'",
 'action',
 'FMESPP',
 'concentrée',
 'financement',
 'mesures',
 'nationales',
 'investissements',
 'missions',
 "d'",
 'expertise',
 'bénéfice',
 'établissements',
 'santé',
 'confi

## Mots

https://chartbeat-labs.github.io/textacy/api_reference.html#textacy.extract.words

In [19]:
from textacy.extract import words

In [20]:
list(words(doc, filter_nums=True, exclude_pos=EXCLUDE_POS))

[article,
 loi,
 décembre,
 financement,
 sécurité,
 fonds,
 modernisation,
 établissements,
 santé,
 FMESPP,
 disparition,
 fonds,
 modernisation,
 transfert,
 obligations,
 créances,
 FMESPP,
 cf,
 art,
 loi,
 décembre,
 financement,
 sécurité,
 Or,
 bilan,
 opération,
 restructuration,
 fonds,
 modernisation,
 législateur,
 établissements,
 services,
 révèle,
 rapport,
 gestion,
 FMESPP,
 Caisse,
 dépôts,
 consignations,
 action,
 FMESPP,
 financement,
 mesures,
 investissements,
 missions,
 expertise,
 établissements,
 santé,
 ATIH,
 ASIP,
 orientation,
 soutien,
 établissements,
 services,
 opérations,
 restructuration,
 rapport,
 p.,
 part,
 financements,
 établissements,
 FMESPP,
 millions,
 euros,
 fonds,
 année,
 millions,
 euros,
 fonds,
 année,
 disproportion,
 secteur,
 secteur,
 niveau,
 établissements,
 services,
 l',
 opérations,
 réorganisation,
 période,
 incitations,
 réorganisations,
 secteur,
 C',
 accès,
 établissements,
 services,
 soutien,
 raison,
 restructurati

# ngrams

https://chartbeat-labs.github.io/textacy/api_reference.html#textacy.extract.ngrams

In [21]:
from textacy.extract import ngrams

In [22]:
list(ngrams(doc, n=1, filter_nums=True, exclude_pos=EXCLUDE_POS))[:20]

[article,
 loi,
 décembre,
 financement,
 sécurité,
 fonds,
 modernisation,
 établissements,
 santé,
 FMESPP,
 disparition,
 fonds,
 modernisation,
 transfert,
 obligations,
 créances,
 FMESPP,
 cf,
 art,
 loi]

In [23]:
list(ngrams(doc, n=2, filter_nums=True, exclude_pos=EXCLUDE_POS))[:20]

[part prise]

### Mots-clés avec _semantic network_

https://chartbeat-labs.github.io/textacy/api_reference.html#textacy.keyterms.key_terms_from_semantic_network

In [24]:
from textacy.keyterms import key_terms_from_semantic_network

In [25]:
key_terms_from_semantic_network(doc, normalize=NORMALIZE)

[('établissements', 0.04252689173147133),
 ('FMESPP', 0.0349351802433892),
 ('secteur', 0.03478361867557645),
 ('privés', 0.03451210302099831),
 ('fonds', 0.025928745316496332),
 ('rapport', 0.01953613255675018),
 ('financement', 0.017373383721947997),
 ('opérations', 0.015813349267153064),
 ('modernisation', 0.015697860647989396),
 ('part', 0.01564399696407981)]

In [26]:
key_terms_from_semantic_network(doc, normalize=NORMALIZE, edge_weighting="cooc_freq")

[('établissements', 0.04781102251776822),
 ('secteur', 0.03746697187549191),
 ('privés', 0.03693875729390274),
 ('fonds', 0.03243644929854286),
 ('FMESPP', 0.03197912312872567),
 ('services', 0.024646611898054598),
 ('modernisation', 0.021978737979084013),
 ('rapport', 0.020139857945073277),
 ('financement', 0.01961927624046945),
 ('financier', 0.01826506203660326)]

In [27]:
key_terms_from_semantic_network(doc, normalize=NORMALIZE, ranking_algo="divrank")

[('établissements', 0.058485334103364726),
 ('secteur', 0.04394348692835522),
 ('FMESPP', 0.04382839871923124),
 ('privés', 0.029286970099640083),
 ('fonds', 0.0267746807181751),
 ('rapport', 0.018879750426097474),
 ('loi', 0.017999891054602958),
 ('financement', 0.013747740647675444),
 ('opérations', 0.013595335065804562),
 ('investissements', 0.012350561036147031)]

In [28]:
key_terms_from_semantic_network(doc, normalize=None, ranking_algo="bestcoverage")

[('établissements', 0.0425289578801367),
 ('FMESPP', 0.03493290830971412),
 ('secteur', 0.03478443420791123),
 ('privés', 0.034512967919346806),
 ('fonds', 0.025929343930338182),
 ('rapport', 0.01953728747034441),
 ('financement', 0.017374944418401345),
 ('opérations', 0.01581390542233655),
 ('modernisation', 0.015697628632571327),
 ('part', 0.015644239227062796)]

## Corpus

Créons un corpus avec les exposés des motifs de tous les amendements :

In [29]:
from textacy import Corpus

def make_corpus(amendements, lang=custom_fr):
    clean_texts = (clean_text(amendement.expose + "\n\n" + amendement.corps) for amendement in amendements)
    return Corpus(lang, clean_texts)

In [30]:
corpus = make_corpus(amendements)

### Mots-clés avec _sgrank_

https://chartbeat-labs.github.io/textacy/api_reference.html#textacy.keyterms.sgrank

In [31]:
from textacy.keyterms import sgrank

On peut optionnellement pondérer les mots clés par rapport à leur fréquence dans l'ensemble du corpus :

In [32]:
idf = corpus.word_doc_freqs(normalize=NORMALIZE)

In [33]:
import random

for _ in range(10):
    doc = random.choice(corpus)
    print(doc.text)
    print("\nsgrank (sans idf) =", [term for term, _ in sgrank(doc, normalize=NORMALIZE, n_keyterms=5)])
    print("\nsgrank (avec idf) =", [term for term, _ in sgrank(doc, normalize=NORMALIZE, idf=idf, n_keyterms=5)])
    print("\n---------------------------------------\n")


Alors que l'article L161‑25 du Code de la Sécurité sociale prévoit l'indexation de l'augmentation des montant des prestations et plafonds de ressources sur l'évolution de la moyenne annuelle des prix à la consommation, l'article 44 du PLFSS pour 2019 prévoit que certains d'entre eux ne soient revalorisés que de 0,3 %. Le niveau de vie des familles n'a cessé de se dégrader. Les familles (aisées, moyennes et modestes) ont subi des ponctions massives et sans précédent depuis 2012 : plus de 4 milliards d'€ d'économies brutes annuelles (modulation des allocations familiales, baisse du plafond du quotient familial, multiples réductions sur la Prestation d'accueil du jeune enfant, etc). Malgré 1 milliard d'euros redistribué par le biais de la revalorisation de prestations sous conditions de ressources, la CNAF dénombrait, en décembre 2016, 3,2 millions de familles avec enfant(s) voyant leur revenu disponible diminuer, dont 60 % sont issus des catégories moyennes et pauvres. Et encore cette ét


sgrank (sans idf) = ['sécurité sociale', 'prélèvements sociaux', 'revenus immobiliers', 'revenus', 'article L.']

sgrank (avec idf) = ['sécurité sociale', 'prélèvements sociaux', 'revenus immobiliers', 'article L.', 'revenus']

---------------------------------------

Les hôpitaux psychiatriques sont financés par une dotation annuelle de financement (DAF), décidée et délivrée par les ARS. Or, contrairement aux hôpitaux dits « classiques », qui voient leurs crédits progresser d'environ 2 % chaque année, cette DAF ne tend pas à évoluer à la hausse. Au contraire, l'enveloppe tend plutôt à diminuer. Un tel mécanisme permet aux ARS de, discrètement, procéder à des économies sur le dos de la psychiatrie et des patients. Pourtant, le nombre de patients augmente régulièrement en psychiatrie. Les statistiques de la Direction de la recherche, des études, de l'évaluation et des statistiques (DREES) en témoignent : + 29 % entre 2013 et 2016. Les urgences débordent : + 36 %. En revanche, les hospi


sgrank (sans idf) = ['niveau régional', 'santé', 'ARS', 'santé publique', 'politiques']

sgrank (avec idf) = ['niveau régional', 'santé publique', 'santé', 'ARS', 'politiques']

---------------------------------------

Les indicateurs actuellement utilisés dans le cadre du dispositif existant IFAQ sont essentiellement des indicateurs de procédure et non des indicateurs de résultats, seuls à même de mesurer la qualité des prestations de soins. Il convient donc, et ce dès 2019, de faire évoluer ces indicateurs et d'intégrer des indicateurs de résultats, y compris des indicateurs de mesure de satisfaction des usagers. Il convient également de veiller à ce que la mesure et l'évaluation de ces indicateurs ne favorisent pas indument des catégories d'établissements afin de garantir l'équité et l'égalité des chances de toutes les structures, quels que soient leurs statuts et leurs activités. Cet amendement vise à garantir, l'équité du dispositif de financement à la qualité. Après l'alinéa 5, 

### Mots-clés avec Vectorizer (TF-IDF)

Créons une matrice (termes ⨉ documents) pondérée par TF-IDF :

In [34]:
from textacy import Vectorizer

vectorizer = Vectorizer(
    tf_type='linear', apply_idf=True, idf_type='smooth', apply_dl=False,
    norm='l2',
    min_df=0,
    max_df=0.9,
)

def doc_to_terms(doc):
    return doc.to_terms_list(as_strings=True, filter_nums=True, normalize=NORMALIZE, exclude_pos=EXCLUDE_POS)

doc_term_matrix = vectorizer.fit_transform((doc_to_terms(doc) for doc in corpus))

doc_term_matrix

<1495x8752 sparse matrix of type '<class 'numpy.float64'>'
	with 93064 stored elements in Compressed Sparse Row format>

Affichons les termes du 1er document :

In [35]:
from itertools import islice
from operator import itemgetter

terms_map = vectorizer.id_to_term

def top_terms(vectorizer, doc, n):
    doc_term_matrix = vectorizer.transform([doc_to_terms(doc)])
    term_ids = doc_term_matrix[0].toarray()[0].tolist()
    sorted_term_ids = sort_term_ids(term_ids)
    sorted_terms = ((terms_map[term_id], weight) for term_id, weight in sorted_term_ids)
    filtered_terms = (term for term, weight in sorted_terms if filter_term(term))
    return list(islice(filtered_terms, 0, n))

def sort_term_ids(term_ids):
    return sorted(enumerate(term_ids), key=itemgetter(1), reverse=True)

def filter_term(term):
    return term and (not term[0].isdigit()) and (term[0] not in {"-", ".", "°", "–", "/"})

for doc in random.sample(corpus.docs, 10):
    print(doc.text)
    print(top_terms(vectorizer, doc, n=5))
    print()

Afin d'améliorer l'équilibre des comptes publics, le Gouvernement a décidé de revaloriser les retraites de 0.3 % seulement, c'est-à-dire moins que l'inflation qui est d'environ 1.2 %. Les retraites augmenteront donc 4 fois moins que le coût de la vie. L'essentiel des efforts que le Gouvernement souhaite faire pour redresser les compte de la Sécurité sociale passe par ce quasi-gel des pensions de retraites. Ajoutez à cela l'augmentation non compensée de la contribution sociale généralisée (CSG) à laquelle ont du faire face les retraités et vous comprenez que l'on pourrait croire que les retraités sont devenus les vaches à lait de la politique gouvernementale. Une étude récente de l'Institut des Politiques Publiques sur les effets de la fiscalité du Gouvernement met en évidence que les retraités ont perdu 1 à 3 % de leur pouvoir d'achat du fait des mesures prises depuis le début du quinquennat. C'est inacceptable et profondément injuste. Car le Gouvernement s'attaque à des gens qui ont t

## Topic Model (WIP)

Créons un _Topic Model_ à partir de cette matrice :

In [36]:
from textacy import TopicModel

model = TopicModel('nmf', n_topics=20)
model.fit(doc_term_matrix)

Calculons une matrice (topics ⨉ documents) :

In [37]:
doc_topic_matrix = model.transform(doc_term_matrix)

In [38]:
#list(model.top_doc_topics(doc_topic_matrix))

# Production d'un CSV pour évaluation manuelle

In [39]:
docs = random.sample(corpus.docs, 500)

In [40]:
def make_row(doc):
    return {
        "text": doc.text,
        "top_terms": "\n".join(top_terms(vectorizer, doc, n=5)),
        "sgrank_idf": "\n".join(term for term, _ in sgrank(doc, normalize=NORMALIZE, idf=idf, n_keyterms=5)),
        "sgrank": "\n".join(term for term, _ in sgrank(doc, normalize=NORMALIZE, n_keyterms=5)),
    }

In [41]:
import csv

with open("tags.csv", "w", newline="\n") as csvfile:
    fieldnames = ["text", "top_terms", "sgrank_idf", "sgrank"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(make_row(doc) for doc in docs)